In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy.stats import entropy

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import classification_report, confusion_matrix 

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)  # 50% dropout
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)  # 50% dropout
        self.fc3 = nn.Linear(hidden_size2, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        out = self.fc3(out)
        return out

# 定义模型参数
input_size = 10 # 输入特征的维度
hidden_size1 = 100 # 第一个隐藏层的大小
hidden_size2 = 50 # 第二个隐藏层的大小
output_size = 2 # 输出的类别数

model = MLP()

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
all_Chinese = pd.read_excel('/Users/anyingbai/Desktop/连续剧/Oxford/COPD模型/最新修改/心血管共病/机器学习模型训练/CHARLS_10_18基线无共病.xlsx', index_col=0)
Chinese2=all_Chinese.copy(deep=True)
Chinese2.dropna(axis=0, how='any', subset=None, inplace=True)
Chinese2.head()
Chinese2.isnull().sum()
Chinese2_Y = Chinese2['Cardiometabolic_multi_18']
Chinese2_X = Chinese2.loc[:,['ADL_Disability','IADL_Disability','edu_group','Male','Married','age','household_wealth','Excessive_drink','physical_activity','smoking_present']]
X_train, X_test, y_train, y_test = train_test_split(Chinese2_X, Chinese2_Y, test_size = 0.40, random_state=42)

In [6]:
import torch.optim as optim
learning_rate = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [7]:

# 定义训练数据加载器
# 可以使用PyTorch的DataLoader来批量加载训练数据
from torch.utils.data import TensorDataset, DataLoader

batch_size = 64
train_dataset = TensorDataset(torch.Tensor(X_train.values), torch.Tensor(y_train.values))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
for batch_data, batch_labels in train_loader:
    batch_data = batch_data.long()
    batch_labels = batch_labels.long()

In [8]:
X_train_tensor = torch.Tensor(X_train.values)
X_test_tensor = torch.Tensor(X_test.values)
y_train_tensor = torch.Tensor(y_train.values)
y_test_tensor = torch.Tensor(y_test.values)

print("X_train Tensor类型:", X_train_tensor.dtype)
print("X_train Tensor设备:", X_train_tensor.device)

print("X_test Tensor类型:", X_test_tensor.dtype)
print("X_test Tensor设备:", X_test_tensor.device)

print("y_train Tensor类型:", y_train_tensor.dtype)
print("y_train Tensor设备:", y_train_tensor.device)

print("y_test Tensor类型:", y_test_tensor.dtype)
print("y_test Tensor设备:", y_test_tensor.device)

X_train Tensor类型: torch.float32
X_train Tensor设备: cpu
X_test Tensor类型: torch.float32
X_test Tensor设备: cpu
y_train Tensor类型: torch.float32
y_train Tensor设备: cpu
y_test Tensor类型: torch.float32
y_test Tensor设备: cpu


In [43]:
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# 使用训练集的均值和标准差对测试集进行标准化
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# 设置模型结构和参数
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # 在这里定义模型结构，使用给定的最佳参数配置
        self.fc = nn.Sequential(
            nn.Linear(10, 50),
            nn.ReLU(),
            nn.Linear(50, 2),
            nn.Sigmoid()  # 在输出层使用 Sigmoid 激活函数
        )

    def forward(self, x):
        x = self.fc(x)
        return x

# 创建模型实例
model = MLP()

# 定义优化器和损失函数
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

# 定义训练和测试函数
def train(model, optimizer, criterion, train_loader):
    model.train()
    train_losses = []
    for batch_data, batch_labels in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels.long())
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    return np.mean(train_losses)

def test(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        val_outputs = model(torch.Tensor(X_test_scaled))
        val_loss = criterion(val_outputs, torch.LongTensor(y_test.values))

        # 计算预测值
        _, predicted = torch.max(val_outputs, 1)
        
        # 计算熵值
        probabilities = F.softmax(val_outputs, dim=1)
        entropy = -torch.sum(probabilities * torch.log2(probabilities), dim=1)  # 计算每个样本的熵值
        
        # 计算平均熵值
        average_entropy = torch.mean(entropy).item()
        
        # 计算评估指标
        precision = precision_score(y_test, predicted)
        recall = recall_score(y_test, predicted)
        f1 = f1_score(y_test, predicted)
        
        # 对于二分类问题，需要使用模型输出的概率值来计算 AUPRC 和 AUROC
        # 获取模型预测的概率值
        probabilities = torch.nn.functional.softmax(val_outputs, dim=1)
        positive_probabilities = probabilities[:, 1].cpu().numpy()  # 正类的概率
        
        # 计算 AUPRC 和 AUROC
        auprc = average_precision_score(y_test, positive_probabilities)
        auroc = roc_auc_score(y_test, positive_probabilities)
        
        return val_loss.item(), precision, recall, f1, auprc, auroc, average_entropy

# 在每个 epoch 中更新测试函数的调用并存储评估指标
test_metrics = {'loss': [], 'precision': [], 'recall': [], 'f1': [], 'auprc': [], 'auroc': [], 'entropy': []}
for epoch in range(num_epochs):
    train_loss = train(model, optimizer, criterion, train_loader)
    test_loss, precision, recall, f1, auprc, auroc, avg_entropy = test(model, X_test_scaled, y_test)  
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    # 存储评估指标
    test_metrics['loss'].append(test_loss)
    test_metrics['precision'].append(precision)
    test_metrics['recall'].append(recall)
    test_metrics['f1'].append(f1)
    test_metrics['auprc'].append(auprc)
    test_metrics['auroc'].append(auroc)
    test_metrics['entropy'].append(avg_entropy)
    
# 输出最终评估指标
final_metrics = {
    'Average_Test_Loss': np.mean(test_metrics['loss']),
    'Average_Precision': np.mean(test_metrics['precision']),
    'Average_Recall': np.mean(test_metrics['recall']),
    'Average_F1': np.mean(test_metrics['f1']),
    'Average_AUPRC': np.mean(test_metrics['auprc']),
    'Average_AUROC': np.mean(test_metrics['auroc']),
    'Average_entropy': np.mean(test_metrics['entropy'])
    
}
print("Final Evaluation Metrics:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value}")

Final Evaluation Metrics:
Average_Test_Loss: 0.6706273257732391
Average_Precision: 0.1469866207429976
Average_Recall: 0.1482570281124498
Average_F1: 0.14016756087741386
Average_AUPRC: 0.16503308565307095
Average_AUROC: 0.43256818708833433
Average_entropy: 0.9961584454774857


In [31]:
#model.eval()
#with torch.no_grad():
    #test_outputs = model(torch.Tensor(X_test_scaled))
#import torch.nn.functional as F

# 计算预测概率分布
#probabilities = F.softmax(test_outputs, dim=1)
#from scipy.stats import entropy

# 计算每个数据点的熵
#entropies = [entropy(probability, base=2) for probability in probabilities.numpy()]

In [32]:
# 假设您已经计算了每个数据点的熵值并存储在了 entropies 列表中
# 将列表转换为 PyTorch 张量
#entropies_tensor = torch.tensor(entropies)

# 计算整个数据集上每个数据点的平均熵
#average_entropy = torch.mean(entropies_tensor)

# 打印整个数据集上每个数据点的平均熵
#print("整个数据集上每个数据点的平均熵：", average_entropy.item())

整个数据集上每个数据点的平均熵： 0.9956860124112181


In [46]:
#引入欧洲数据
all_Europe = pd.read_excel('/Users/anyingbai/Desktop/连续剧/Oxford/COPD模型/最新修改/心血管共病/机器学习模型训练/SHARE_10_18基线无共病.xlsx', index_col=0)
Europe2=all_Europe.copy(deep=True)
Europe2.dropna(axis=0, how='any', subset=None, inplace=True)
Europe2.head()
Europe2_Y = Europe2['Cardiometabolic_multi_18']
Europe2_X = Europe2.loc[:,['ADL_Disability','IADL_Disability','edu_group','Male','Married','age','household_wealth','Excessive_drink','physical_activity','smoking_present']]
X_train, X_test, y_train, y_test = train_test_split(Europe2_X, Europe2_Y, test_size = 0.40, random_state=42)

In [49]:
from sklearn.preprocessing import StandardScaler
# 使用训均值和标准差对测试集进行标准化
scaler = StandardScaler()
Europe2_X_scaled = scaler.fit_transform(Europe2_X)

def test(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        val_outputs = model(torch.Tensor(Europe2_X_scaled))
        val_loss = criterion(val_outputs, torch.LongTensor(Europe2_Y.values))

        # 计算预测值
        _, predicted = torch.max(val_outputs, 1)
        
        # 计算熵值
        probabilities = F.softmax(val_outputs, dim=1)
        entropy = -torch.sum(probabilities * torch.log2(probabilities), dim=1)  # 计算每个样本的熵值
        
        # 计算平均熵值
        average_entropy = torch.mean(entropy).item()
        
        # 计算评估指标
        precision = precision_score(Europe2_Y, predicted)
        recall = recall_score(Europe2_Y, predicted)
        f1 = f1_score(Europe2_Y, predicted)
        
        # 对于二分类问题，需要使用模型输出的概率值来计算 AUPRC 和 AUROC
        # 获取模型预测的概率值
        probabilities = torch.nn.functional.softmax(val_outputs, dim=1)
        positive_probabilities = probabilities[:, 1].cpu().numpy()  # 正类的概率
        
        # 计算 AUPRC 和 AUROC
        auprc = average_precision_score(Europe2_Y, positive_probabilities)
        auroc = roc_auc_score(Europe2_Y, positive_probabilities)
        
        return val_loss.item(), precision, recall, f1, auprc, auroc, average_entropy

# 在每个 epoch 中更新测试函数的调用并存储评估指标
test_metrics = {'loss': [], 'precision': [], 'recall': [], 'f1': [], 'auprc': [], 'auroc': [], 'entropy': []}
for epoch in range(num_epochs):
    train_loss = train(model, optimizer, criterion, train_loader)
    test_loss, precision, recall, f1, auprc, auroc, avg_entropy = test(model, Europe2_X_scaled, Europe2_Y)  
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    # 存储评估指标
    test_metrics['loss'].append(test_loss)
    test_metrics['precision'].append(precision)
    test_metrics['recall'].append(recall)
    test_metrics['f1'].append(f1)
    test_metrics['auprc'].append(auprc)
    test_metrics['auroc'].append(auroc)
    test_metrics['entropy'].append(avg_entropy)
    
# 输出最终评估指标
final_metrics = {
    'Average_Test_Loss': np.mean(test_metrics['loss']),
    'Average_Precision': np.mean(test_metrics['precision']),
    'Average_Recall': np.mean(test_metrics['recall']),
    'Average_F1': np.mean(test_metrics['f1']),
    'Average_AUPRC': np.mean(test_metrics['auprc']),
    'Average_AUROC': np.mean(test_metrics['auroc']),
    'Average_entropy': np.mean(test_metrics['entropy'])
    
}
print("Final Evaluation Metrics:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value}")

Final Evaluation Metrics:
Average_Test_Loss: 0.6341543853282928
Average_Precision: 0.1474660404162232
Average_Recall: 0.008038922155688626
Average_F1: 0.01523538215133514
Average_AUPRC: 0.11121311417237635
Average_AUROC: 0.41538929699343535
Average_entropy: 0.9887345224618912


In [36]:
#引入美国数据

In [37]:
all_US = pd.read_excel('/Users/anyingbai/Desktop/连续剧/Oxford/COPD模型/最新修改/心血管共病/机器学习模型训练/HRS_10_18基线无共病.xlsx', index_col=0)
US2=all_US.copy(deep=True)
US2.dropna(axis=0, how='any', subset=None, inplace=True)
US2.head()

,Male,edu_group,Married,age,smoking_present,physical_activity,household_wealth,ADL_Disability,IADL_Disability,Excessive_drink,Cardiometabolic_multi_18
ID,,,,,,,,,,,
10004040,0,1.0,1.0,64,0.0,1.0,103050000,0,0,1,0
10004040,0,1.0,1.0,64,0.0,1.0,103050000,0,0,1,0
10004040,0,1.0,1.0,64,0.0,1.0,103050000,0,0,1,0
10004040,0,1.0,1.0,64,0.0,1.0,103050000,0,0,1,0
10004040,0,1.0,1.0,64,0.0,1.0,103050000,0,0,1,0


In [38]:
US2_Y = US2['Cardiometabolic_multi_18']
US2_X = US2.loc[:,['ADL_Disability','IADL_Disability','edu_group','Male','Married','age','household_wealth','Excessive_drink','physical_activity','smoking_present']]

In [50]:
from sklearn.preprocessing import StandardScaler
# 使用训均值和标准差对测试集进行标准化
scaler = StandardScaler()
US2_X_scaled = scaler.fit_transform(US2_X)

def test(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        val_outputs = model(torch.Tensor(US2_X_scaled))
        val_loss = criterion(val_outputs, torch.LongTensor(US2_Y.values))

        # 计算预测值
        _, predicted = torch.max(val_outputs, 1)
        
        # 计算熵值
        probabilities = F.softmax(val_outputs, dim=1)
        entropy = -torch.sum(probabilities * torch.log2(probabilities), dim=1)  # 计算每个样本的熵值
        
        # 计算平均熵值
        average_entropy = torch.mean(entropy).item()
        
        # 计算评估指标
        precision = precision_score(US2_Y, predicted)
        recall = recall_score(US2_Y, predicted)
        f1 = f1_score(US2_Y, predicted)
        
        # 对于二分类问题，需要使用模型输出的概率值来计算 AUPRC 和 AUROC
        # 获取模型预测的概率值
        probabilities = torch.nn.functional.softmax(val_outputs, dim=1)
        positive_probabilities = probabilities[:, 1].cpu().numpy()  # 正类的概率
        
        # 计算 AUPRC 和 AUROC
        auprc = average_precision_score(US2_Y, positive_probabilities)
        auroc = roc_auc_score(US2_Y, positive_probabilities)
        
        return val_loss.item(), precision, recall, f1, auprc, auroc, average_entropy

# 在每个 epoch 中更新测试函数的调用并存储评估指标
test_metrics = {'loss': [], 'precision': [], 'recall': [], 'f1': [], 'auprc': [], 'auroc': [], 'entropy': []}
for epoch in range(num_epochs):
    train_loss = train(model, optimizer, criterion, train_loader)
    test_loss, precision, recall, f1, auprc, auroc, avg_entropy = test(model, US2_X_scaled, US2_Y)  
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    # 存储评估指标
    test_metrics['loss'].append(test_loss)
    test_metrics['precision'].append(precision)
    test_metrics['recall'].append(recall)
    test_metrics['f1'].append(f1)
    test_metrics['auprc'].append(auprc)
    test_metrics['auroc'].append(auroc)
    test_metrics['entropy'].append(avg_entropy)
    
# 输出最终评估指标
final_metrics = {
    'Average_Test_Loss': np.mean(test_metrics['loss']),
    'Average_Precision': np.mean(test_metrics['precision']),
    'Average_Recall': np.mean(test_metrics['recall']),
    'Average_F1': np.mean(test_metrics['f1']),
    'Average_AUPRC': np.mean(test_metrics['auprc']),
    'Average_AUROC': np.mean(test_metrics['auroc']),
    'Average_entropy': np.mean(test_metrics['entropy'])
    
}
print("Final Evaluation Metrics:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value}")

Final Evaluation Metrics:
Average_Test_Loss: 0.6370086723566055
Average_Precision: 0.1781131411875776
Average_Recall: 0.023665968079961307
Average_F1: 0.041779166197829225
Average_AUPRC: 0.17342077204096737
Average_AUROC: 0.4452768954321207
Average_entropy: 0.9842941296100617
